# LED

## 패턴
왼쪽으로 : pattern <<= 1;  
오른쪽으로 : pattern >>= 1;  
왼쪽으로 채우기 :  pattern <<= 1; pattern |= 0x01;  
오른쪽으로 채우기 : pattern >>= 1; pattern |= 0x80;   

# 스위치

## four mode  
sw1 = (PINE&0x30)==0x20)  
sw2 = (PINE&0x30)==0x10)  
sw1 and sw2 = ((PINE&0x30)==0x30)   
off = ((PINE&0x30)==0x00)  

### 누르고 있으면 계속 바뀜
```
// CPU 클럭 주파수 정의
#define F_CPU 16000000UL

// 필요한 AVR 라이브러리 포함
#include <avr/io.h>
#include <util/delay.h>

// 메인 함수
int main(void)
{
    // 패턴을 나타내는 변수 초기화
    char pattern = 0x01;
    //    char pattern = 0x80; // 초기 패턴을 오른쪽에서 시작하도록 변경

    // 포트 A를 출력으로 설정
    DDRA = 0xFF;

    // 포트 E를 입력으로 설정
    DDRE = 0x00;

    // 초기 패턴으로 포트 A 초기화
    PORTA = pattern;

    // 무한 루프
    while (1)
    {
        // 포트 E에서 지정된 비트가 0x20과 같은지 확인
        if ((PINE & 0x30) == 0x20)
        {
            // 패턴을 왼쪽으로 1비트 이동
            pattern <<= 1;

            // 패턴이 0x00이 되면 0x01로 리셋
            if (pattern == 0x00)
                pattern = 0x01;

            // 새로운 패턴으로 포트 A 업데이트
            PORTA = pattern;
        }

        // 100 밀리초 동안 대기
        _delay_ms(100);
    }

    // 이 부분은 실행되지 않아야 함
    return 0;
}

```

### 누른 순간에 시프트
~~~
// CPU 클럭 주파수 정의
#define F_CPU 16000000UL

// 필요한 AVR 라이브러리 포함
#include <avr/io.h>
#include <util/delay.h>

// 이전 스위치 상태를 저장할 변수 초기화
char prev_switch_state = 0;

// 메인 함수
int main(void)
{
    // 패턴을 나타내는 변수 초기화
    char pattern = 0x01;

    // 포트 A를 출력으로 설정
    DDRA = 0xFF;

    // 포트 E를 입력으로 설정
    DDRE = 0x00;

    // 초기 패턴으로 포트 A 초기화
    PORTA = pattern;

    // 무한 루프
    while (1)
    {
        // 현재 스위치 상태 읽기
        char switch_state = PINE & 0x30;

        // 스위치가 눌러진 순간 (전환되었을 때)
        if (switch_state != prev_switch_state)
        {
            // 스위치가 눌러진 경우
            if (switch_state == 0x20)
            {
                // 패턴을 왼쪽으로 1비트 이동
                pattern <<= 1;

                // 패턴이 0x00이 되면 0x01로 리셋
                if (pattern == 0x00)
                    pattern = 0x01;

                // 새로운 패턴으로 포트 A 업데이트
                PORTA = pattern;
            }

            // 이전 스위치 상태 업데이트
            prev_switch_state = switch_state;
        }

        // 100 밀리초 동안 대기
        _delay_ms(100);
    }


    return 0;
}
~~~


### 누른 순간에 시프트 단순버전
~~~
// CPU 클럭 주파수 정의
#define F_CPU 16000000UL

// 필요한 AVR 라이브러리 포함
#include <avr/io.h>
#include <util/delay.h>

int main(void)
{
    char pattern = 0x01;

    // 포트 A를 출력으로 설정
    DDRA = 0xFF;

    // 포트 E를 입력으로 설정
    DDRE = 0x00;

    // 초기 패턴으로 포트 A 초기화
    PORTA = pattern;

    while (1)
    {
        // 버튼이 눌렸는지 확인
        if ((PINE & 0x20) == 0x20)
        {
            // 패턴을 왼쪽으로 1비트 시프트
            pattern <<= 1;

            // 패턴이 0x00이 되면 0x01로 리셋
            if (pattern == 0x00)
                pattern = 0x01;

            // 새로운 패턴으로 포트 A 업데이트
            PORTA = pattern;

            // 버튼 떼기까지 대기 (반복을 한 번만 수행하도록)
            while ((PINE & 0x20) == 0x20);
        }
    }
}


### 뗀 순간에 시프트 
#define F_CPU 16000000UL
#include <avr/io.h>
#include <util/delay.h>


int main(void)
{
    char pattern = 0x01;

    DDRA = 0xFF;        // Set PORTA as output
    DDRE = 0x00;        // Set PORTE as input
    PORTE |= 0x30;      // Enable internal pull-up resistors for PORTE 4 and 5

    while (1)
    {
        // Check if the switch is pressed
        if ((PINE & 0x20) == 0x20)
        {
            // Stay inside this while loop as long as the switch is pressed
            while ((PINE & 0x20) == 0x20)
            {
                // Do nothing, just wait for the switch to be released
            }
            
            // Switch is released, move the pattern to the left
            pattern <<= 1;
            if (pattern == 0x00)
                pattern = 0x01;
            PORTA = pattern;
            
            _delay_ms(100);  // Debounce delay
        }
    }

    return 0;
}



# 시리얼 통신

### 숫자 -> 빛

~~~





#define F_CPU 16000000UL
#include <avr/io.h>
#include <util/delay.h>
#include "UART0.h"

int main(void)
{
    char data;
    UART0_init();
    DDRA = 0xff;
    PORTA = 0x00;

    while (1) {
        // 데이터 수신
        data = UART0_receive();
        
        // 수신한 데이터 송신
        UART0_transmit(data);
        
        // 데이터에 따른 PORTA 값 설정
        if (data == '0') PORTA = 0x00;
        else if (data == '1') PORTA = 0x01;
        else if (data == '2') PORTA = 0x03;
        else if (data == '3') PORTA = 0x07;
        else if (data == '4') PORTA = 0x0F;
        else if (data == '5') PORTA = 0x1F;
        else if (data == '6') PORTA = 0x3F;
        else if (data == '7') PORTA = 0x7F;
        else if (data == '8') PORTA = 0xFF;
    }
}















#define F_CPU 16000000UL
#include <avr/io.h>
#include <util/delay.h>
#include "UART0.h"

int main(void)
{
    char data;
    int num;

    UART0_init();
    DDRA = 0xff;
    PORTA = 0x00;

    while (1) {
        // 데이터 수신
        data = UART0_receive();
        
        // 수신한 데이터 송신
        UART0_transmit(data);
        
        // 문자를 숫자로 변환
        num = data - '0';

        // 숫자에 따른 PORTA 값 설정
        if (num == 0) PORTA = 0x00;
        else if (num == 1) PORTA = 0x01;
        else if (num == 2) PORTA = 0x03;
        else if (num == 3) PORTA = 0x07;
        else if (num == 4) PORTA = 0x0F;
        else if (num == 5) PORTA = 0x1F;
        else if (num == 6) PORTA = 0x3F;
        else if (num == 7) PORTA = 0x7F;
        else if (num == 8) PORTA = 0xFF;
    }
}


## type up or down to contorl the numbers

~~~





#define F_CPU 16000000UL
#include <avr/io.h>
#include <util/delay.h>
#include "UART0.h"
#include <string.h>
#include <stdio.h>

// 표준 입출력 스트림 설정
FILE OUTPUT = FDEV_SETUP_STREAM(UART0_transmit, NULL, _FDEV_SETUP_WRITE);
FILE INPUT = FDEV_SETUP_STREAM(NULL, UART0_receive, _FDEV_SETUP_READ);

int main(void)
{
    // 카운터 변수 및 버퍼 배열 초기화
    uint8_t counter = 100;
    char buffer[20] = "";

    // 표준 입출력 스트림 설정
    stdout = &OUTPUT;
    stdin = &INPUT;

    // UART 초기화
    UART0_init();

    // 현재 카운터 값 출력
    printf("현재 카운터 값: ");
    printf("%d\r\n", counter);

    while (1)
    {
        // 문자열 입력 받기
        scanf("%s", buffer);

        // 입력된 문자열과 "DOWN" 비교
        if (strcasecmp(buffer, "DOWN") == 0)
        {
            // 카운터 감소 및 현재 값 출력
            counter--;
            printf("현재 카운터 값: ");
            printf("%d\r\n", counter);
        }
        // 입력된 문자열과 "UP" 비교
        else if (strcasecmp(buffer, "UP") == 0)
        {
            // 카운터 증가 및 현재 값 출력
            counter++;
            printf("현재 카운터 값: ");
            printf("%d\r\n", counter);
        }
        // 알 수 없는 명령일 경우 메시지 출력
        else
        {
            printf("** 알 수 없는 명령 **\r\n");
        }
    }
}


# 모터

## 속도 증가

~~~


#define F_CPU 16000000UL
#include <avr/io.h>
#include <util/delay.h>
#define SpeedMotor1(s) OCR1B = s
void InitializeTimer1(void)
{
TCCR1A |= (1<<WGM10); //8비트 고속 PWM모드
TCCR1B |= (1<<WGM12); //8비트 고속 PWM모드
TCCR1A |= (1<<COM1B1); //비반전모드
TCCR1B |= (1<<CS11) | (1<<CS10); //분주비 64
OCR1B = 0;
}

int main(void)
{
DDRB = 0xff;
PORTB = 0x00;
InitializeTimer1();
while(1){
for(int i = 0; i <256; i++){
SpeedMotor1(i);
_delay_ms(25);
}
SpeedMotor1(0);
_delay_ms(1000);
}
return 0;
}

## 모터 + 분주비 + 방향

코드 : 
1. 시계 방향으로 최저속도에서 최고속도로 회전
2. 1초 동안 정지
3. 반시계방향으로 최저속도에서 최고속도로 회전
4.  1초 동안 정지
5.  타이머/카운터1의 PWM의 8비트 고속 PWM 사용




~~~
#include <avr/io.h>       // AVR 라이브러리를 포함
#include <util/delay.h>   // 딜레이 함수 라이브러리를 포함

#define F_CPU 16000000UL  // CPU 클럭 주파수를 16MHz로 정의

// 모터 1 속도 설정 매크로
#define SpeedMotor1(s) OCR1B = s
// 모터 2 속도 설정 매크로
#define SpeedMotor2(s) OCR1C = s

// 타이머1 초기화 함수
void InitializeTimer1(void)
{
    // 타이머1 설정
    TCCR1A |= (1<<WGM10);  // 8비트 고속 PWM 모드 선택
    TCCR1B |= (1<<WGM12);  // 8비트 고속 PWM 모드 선택
    TCCR1A |= (1<<COM1B1); // 비반전 모드로 모터 1 출력 설정
    TCCR1A |= (1<<COM1C1); // 비반전 모드로 모터 2 출력 설정

    TCCR1B = 0; // 프리스케일러를 0으로 설정 (타이머를 멈춤)
    TCCR1B |= (1<<CS10); // 프리스케일러를 1로 설정 (클럭 소스를 나누지 않음)
    TCCR1B |= (1<<CS11); // 프리스케일러를 8로 설정
    TCCR1B |= (1<<CS11) | (1<<CS10); // 프리스케일러를 64로 설정
    TCCR1B |= (1<<CS12); // 프리스케일러를 256으로 설정
    TCCR1B |= (1<<CS12) | (1<<CS10); // 프리스케일러를 1024로 설정
    
    OCR1B = 0;  // 모터 1 속도 초기화
    OCR1C = 0;  // 모터 2 속도 초기화
}

int main(void)
{
    DDRB = 0xff;   // 포트 B의 모든 핀을 출력으로 설정
    PORTB = 0x00;  // 포트 B의 출력 초기화
    InitializeTimer1();  // 타이머1 초기화
    
    while(1)
    {
        // 모터 1 속도 증가
        for(int i = 0; i < 256; i++)
        {
            SpeedMotor1(i);     // 모터 1 속도 설정
            _delay_ms(25);      // 25ms 대기
        }
        SpeedMotor1(0);         // 모터 1 정지
        _delay_ms(1000);        // 1초 대기
        
        // 모터 2 속도 증가
        for(int i = 0; i < 256; i++)
        {
            SpeedMotor2(i);     // 모터 2 속도 설정
            _delay_ms(25);      // 25ms 대기
        }
        SpeedMotor2(0);         // 모터 2 정지
        _delay_ms(1000);        // 1초 대기
    }
    
    return 0;
}
~~~

## 버튼 눌러서 속도 조절
~~~




#define F_CPU 16000000UL
#include <avr/io.h>
#include <util/delay.h>
#define SpeedMotor1(s) OCR1B = s

void InitializeTimer1(void) {
    TCCR1A |= (1<<WGM10); //8비트 고속 PWM모드
    TCCR1B |= (1<<WGM12); //8비트 고속 PWM모드
    TCCR1A |= (1<<COM1B1); //비반전모드
    TCCR1B |= (1<<CS11) | (1<<CS10); //분주비 64
    OCR1B = 0;
}

int main(void) {
    DDRB = 0xff; // PORTB를 출력으로 설정
    PORTB = 0x00; // PORTB 초기화
    DDRE &= ~(0x30); // PINE 4, 5번 핀을 입력으로 설정
    PORTE |= 0x30; // PINE 4, 5번 핀에 풀업 저항 활성화

    InitializeTimer1();
    int speed = 0;
    
    while(1) {
        if((PINE & 0x30) == 0x20) { // 첫 번째 버튼이 눌렸을 때
            if(speed < 255) speed += 5; // 속도 증가
            _delay_ms(250); // 버튼 디바운스
        }
        else if((PINE & 0x30) == 0x10) { // 두 번째 버튼이 눌렸을 때
            if(speed > 0) speed -= 5; // 속도 감소
            _delay_ms(250); // 버튼 디바운스
        }

        SpeedMotor1(speed);
    }
    
    return 0;
}


# 빛감지 내장

## 100 이상 / 이하 led조절




~~~




#include <stdio.h>
#include <avr/io.h>
#include <util/delay.h>
#include "UART0.h"

int main(void)
{
    // 표준 입출력 설정
    stdout = &OUTPUT;
    stdin = &INPUT;

    int read;
    int CDS_ref = 0;

    // PORTA 설정
    DDRA = 0xff;
    PORTA = 0x00;

    // UART 초기화
    UART0_init();

    // ADC 초기화
    ADC_init(0);

    // CDS_ref 측정을 위한 10회 반복 측정 및 평균 계산
    for (int i = 0; i < 10; i++) {
        CDS_ref += read_ADC();
    }
    CDS_ref = CDS_ref / 10;

    // CDS_ref 값 출력
    printf("CDS_ref: ");
    printf("%d\r\n", CDS_ref);
    _delay_ms(2000);

    while (1) {
        // ADC로부터 값 읽기
        read = read_ADC();
        
        // 읽은 값 출력
        printf("%d\r\n", read);

        // 조건에 따라 PORTA 설정
        if (read < CDS_ref - 100)
            PORTA = 0xff;
        else
            PORTA = 0x00;
    }
}


#### ADC를 시리얼로 출력 // 프리러닝

~~~
// CPU 클럭 주파수를 16MHz로 정의
#define F_CPU 16000000UL

// 필요한 AVR 라이브러리와 헤더 파일을 포함
#include <avr/io.h>
#include <util/delay.h>
#include "UART0.h" // UART 헤더 파일
#include <string.h>
#include <stdio.h>

// 출력 및 입력 파일 스트림 설정
FILE OUTPUT = FDEV_SETUP_STREAM(UART0_transmit, NULL, _FDEV_SETUP_WRITE);
FILE INPUT = FDEV_SETUP_STREAM(NULL, UART0_receive, _FDEV_SETUP_READ);

// ADC 초기화 함수
void ADC_init(unsigned char channel)
{
    ADMUX |= (1 << REFS0); // ADC 참조 전압을 AVCC로 설정
    ADCSRA |= 0x07; // ADC 분주비를 128로 설정
    ADCSRA |= (1 << ADEN); // ADC 활성화
    ADCSRA |= (1 << ADFR); // 자동 트리거 모드 활성화
    ADMUX = ((ADMUX & 0xE0) | channel); // 입력 채널 선택
    ADCSRA |= (1 << ADSC); // ADC 변환 시작
}

// ADC 값을 읽는 함수
int read_ADC(void)
{
    while (!(ADCSRA & (1 << ADIF))); // ADC 변환 완료 대기
    return ADC; // ADC 값을 반환
}

int main(void)



{
    stdout = &OUTPUT; // 표준 출력 스트림 설정
    stdin = &INPUT; // 표준 입력 스트림 설정
    int read;
    
    UART0_init(); // UART 초기화 함수 호출
    ADC_init(0); // ADC 초기화 함수 호출
    
    while (1)
    {
        read = read_ADC(); // ADC 값을 읽음
        printf("%d\r\n", read); // 읽은 ADC 값을 시리얼로 출력
        _delay_ms(1000); // 1초 대기
    }
    
    return 0;
}


단일 변환 모드는 변환마다 명령을 내려야 하기 때문에 특정 이벤트나 조건에 따라 변환을 수행할 때 유용하며,


 프리러닝 모드는 주기적인 데이터 획득이 필요한 상황에서 유용합니다. 

### 빛감지 단일변환모드
~~~
// CPU 클럭 주파수를 16MHz로 정의
#define F_CPU 16000000UL

// 필요한 AVR 라이브러리와 헤더 파일을 포함
#include <avr/io.h>
#include <util/delay.h>
#include "UART0.h" // UART 헤더 파일
#include <string.h>
#include <stdio.h>

// 출력 및 입력 파일 스트림 설정
FILE OUTPUT = FDEV_SETUP_STREAM(UART0_transmit, NULL, _FDEV_SETUP_WRITE);
FILE INPUT = FDEV_SETUP_STREAM(NULL, UART0_receive, _FDEV_SETUP_READ);

// ADC 초기화 함수
void ADC_init()
{
    ADMUX |= (1 << REFS0); // ADC 참조 전압을 AVCC로 설정
    ADCSRA |= 0x07; // ADC 분주비를 128로 설정
    ADCSRA |= (1 << ADEN); // ADC 활성화
}

// ADC 값을 읽는 함수
int read_ADC(unsigned char channel)
{
    ADMUX = ((ADMUX & 0xE0) | channel); // 입력 채널 선택
    ADCSRA |= (1 << ADSC); // ADC 변환 시작
    while (!(ADCSRA & (1 << ADIF))); // ADC 변환 완료 대기
    return ADC; // ADC 값을 반환
}

int main(void)
{
    stdout = &OUTPUT; // 표준 출력 스트림 설정
    stdin = &INPUT; // 표준 입력 스트림 설정
    int read;
    
    UART0_init(); // UART 초기화 함수 호출
    ADC_init(); // ADC 초기화 함수 호출
    
    while (1)
    {
        read = read_ADC(0); // ADC 채널 0에서 값을 읽음
        printf("%d\r\n", read); // 읽은 ADC 값을 시리얼로 출력
        _delay_ms(1000); // 1초 대기
    }
    
    return 0;
}


## ADC인터럽트
~~~


#define F_CPU 16000000UL
#include <avr/io.h>
#include <util/delay.h>
#include "UART0.h"
#include <string.h>
#include <stdio.h>
#include <avr/interrupt.h>

// 표준 입출력 스트림 설정
FILE OUTPUT = FDEV_SETUP_STREAM(UART0_transmit, NULL, _FDEV_SETUP_WRITE);
FILE INPUT = FDEV_SETUP_STREAM(NULL, UART0_receive, _FDEV_SETUP_READ);

// ADC 값을 저장할 변수 선언
volatile unsigned int adc_value;

// ADC 초기화 함수 정의
void ADC_init(unsigned char channel)
{
    // ADC 참조 전압 설정
    ADMUX |= (1 << REFS0);
    
    // ADC 프리스케일러 분주비 설정
    ADCSRA |= 0x07;
    
    // ADC 활성화
    ADCSRA |= (1 << ADEN);
    
    // ADC 자동 트리거 모드 설정
    ADCSRA |= (1 << ADFR);
    
    // ADC 인터럽트 활성화
    ADCSRA |= (1 << ADIE);
    
    // ADMUX에서 채널 설정
    ADMUX = ((ADMUX & 0xE0) | channel);
    
    // ADC 변환 시작
    ADCSRA |= (1 << ADSC);
    
    // 전역 인터럽트 활성화
    sei();
}

// ADC 변환 완료 인터럽트 서비스 루틴
ISR(ADC_vect)
{
    adc_value = ADC;
}

int main(void)
{
    // 표준 입출력 스트림 설정
    stdout = &OUTPUT;
    stdin = &INPUT;
    
    // UART 초기화
    UART0_init();
    
    // ADC 초기화 (채널 1 선택)
    ADC_init(1);
    
    while (1)
    {
        // ADC 값 출력
        printf("%d\r\n", adc_value);
        
        // 500ms 대기
        _delay_ms(500);
    }
}


## 빛감지 인터럽트
~~~
#define F_CPU 16000000UL
#include <avr/io.h>
#include <util/delay.h>
#include "UART0.h"
#include <stdio.h>
#include <avr/interrupt.h>

volatile unsigned int adc_value;

void ADC_init(unsigned char channel)
{
    // Set reference voltage to AVCC with external capacitor at AREF pin.
    ADMUX = (1 << REFS0);
    // Enable ADC, enable ADC interrupt, set ADC prescaler to 128.
    ADCSRA = (1 << ADEN) | (1 << ADIE) | (1 << ADPS2) | (1 << ADPS1) | (1 << ADPS0);
    // Select the ADC channel.
    ADMUX = (ADMUX & 0xF8) | (channel & 0x07);
    // Start the conversion.
    ADCSRA |= (1 << ADSC);
}

ISR(ADC_vect)
{
    adc_value = ADC;
}

int main(void)
{
    // Initialize UART and ADC.
    UART0_init();
    ADC_init(0);

    // Enable global interrupts.
    sei();

    while (1)
    {
        // Print the ADC value over UART.
        printf("%d\r\n", adc_value);
        _delay_ms(1000);
    }

    return 0;
}
~~~

## 빛감지 모듈



#### ****
#### **** <-   a0

### 프리러닝
~~~
#define F_CPU 16000000UL
#include <avr/io.h>
#include <util/delay.h>
#include <stdio.h>
#include <avr/interrupt.h>

// UART 관련 설정
#include "UART0.h"
FILE OUTPUT = FDEV_SETUP_STREAM(UART0_transmit, NULL, _FDEV_SETUP_WRITE);
FILE INPUT = FDEV_SETUP_STREAM(NULL, UART0_receive, _FDEV_SETUP_READ);

// ADC 초기화 함수
void ADC_init(unsigned char channel)
{
    // AREF를 AVCC로 선택
    ADMUX |= (1 << REFS0);
    
    // 프리스케일러 분주비를 128로 설정
    ADCSRA |= (1 << ADPS2) | (1 << ADPS1) | (1 << ADPS0);


    // 분주비 2 설정
    ADCSRA |= (1 << ADPS0); // 2

    // 분주비 4 설정
    ADCSRA |= (1 << ADPS1); // 4

    // 분주비 8 설정
    ADCSRA |= (1 << ADPS1) | (1 << ADPS0); // 8

    // 분주비 16 설정
    ADCSRA |= (1 << ADPS2); // 16

    // 분주비 32 설정
    ADCSRA |= (1 << ADPS2) | (1 << ADPS0); // 32

    // 분주비 64 설정
    ADCSRA |= (1 << ADPS2) | (1 << ADPS1); // 64

    // 분주비 128 설정
    ADCSRA |= (1 << ADPS2) | (1 << ADPS1) | (1 << ADPS0); // 128



    
    // ADC 활성화
    ADCSRA |= (1 << ADEN);
    
    // 자동 트리거 모드 활성화
    ADCSRA |= (1 << ADATE);
    
    // 채널 선택
    ADMUX = ((ADMUX & 0xF0) | (channel & 0x0F));
    
    // 변환 시작
    ADCSRA |= (1 << ADSC);
}

// ADC 값을 읽는 함수
int read_ADC(void)
{
    while (!(ADCSRA & (1 << ADIF))); // 변환 완료까지 대기
    ADCSRA |= (1 << ADIF); // ADIF 비트를 설정하여 인터럽트 플래그 클리어
    return ADC; // ADC 레지스터의 값을 반환
}

int main(void)
{
    stdout = &OUTPUT;
    stdin = &INPUT;
    
    int read;
    
    // UART 및 ADC 초기화
    UART0_init();
    ADC_init(1); // 채널 1을 사용하여 ADC 초기화
    
    // 무한 루프
    while (1)
    {
        read = read_ADC(); // ADC 값을 읽어옴
        printf("%d\r\n", read); // UART로 출력
        
        _delay_ms(500); // 500ms 지연
    }
}
~~~

### 단일변환모드

~~~
#define F_CPU 16000000UL
#include <avr/io.h>
#include <util/delay.h>
#include "UART0.h"
#include <string.h>
#include <stdio.h>
FILE OUTPUT = FDEV_SETUP_STREAM(UART0_transmit, NULL, _FDEV_SETUP_WRITE);
FILE INPUT = FDEV_SETUP_STREAM(NULL, UART0_receive, _FDEV_SETUP_READ);

// ADC 초기화 함수
void ADC_init()
{
    ADMUX |= (1<<REFS0); // 기준전압을 AVCC로 설정
    ADCSRA |= 0x07; // 분주비 설정 (prescaler 128)
    ADCSRA |= (1<<ADEN); // ADC 활성화
}

// ADC 값을 읽는 함수
int read_ADC(unsigned char channel)
{
    ADMUX = ((ADMUX & 0xE0) | channel); // 채널 선택
    ADCSRA |= (1<<ADSC); // 변환 시작
    while(!(ADCSRA & (1<<ADIF))); // 변환 완료까지 대기
    return ADC; // 변환된 ADC 값을 반환
}

int main(void)
{
    stdout = &OUTPUT;
    stdin = &INPUT;
    int read;
    UART0_init(); // UART 초기화
    ADC_init(); // ADC 초기화
    while (1)
    {
        read = read_ADC(1); // 채널 1에서 ADC 값을 읽어옴
        printf("%d\r\n", read); // UART를 통해 값을 출력
        _delay_ms(500); // 500ms 지연
    }
}
~~~

### 인터럽트

~~~
#define F_CPU 16000000UL
#include <avr/io.h>
#include <util/delay.h>
#include "UART0.h"
#include <string.h>
#include <stdio.h>
#include <avr/interrupt.h>

FILE OUTPUT = FDEV_SETUP_STREAM(UART0_transmit, NULL, _FDEV_SETUP_WRITE);
FILE INPUT = FDEV_SETUP_STREAM(NULL, UART0_receive, _FDEV_SETUP_READ);

volatile unsigned int adc_value;

// ADC 초기화 함수
void ADC_init(unsigned char channel)
{
    ADMUX |= (1<<REFS0); // 기준전압을 AVCC로 설정
    ADCSRA |= 0x07; // 분주비 설정 (prescaler 128)
    ADCSRA |= (1<<ADEN); // ADC 활성화
    ADCSRA |= (1<<ADFR); // 자동 트리거 모드 활성화
    ADCSRA |= (1<<ADIE); // ADC 변환 완료 인터럽트 활성화
    ADMUX = ((ADMUX & 0xE0) | channel); // 채널 선택
    ADCSRA |= (1<<ADSC); // 변환 시작
    sei(); // 전역 인터럽트 활성화
}

// ADC 변환 완료 인터럽트 핸들러
ISR(ADC_vect)
{
    adc_value = ADC; // ADC 값 저장
}

int main(void)
{
    stdout = &OUTPUT;
    stdin = &INPUT;
    UART0_init(); // UART 초기화
    ADC_init(1); // 채널 1에서 ADC 초기화
    while (1)
    {
        printf("%d\r\n", adc_value); // UART를 통해 ADC 값을 출력
        _delay_ms(500); // 500ms 지연
    }
}
~~~

### 입력전압 계산 3V >



~~~
#define F_CPU 16000000UL  // 마이크로컨트롤러의 CPU 클럭 주파수를 16MHz로 정의

#include <avr/io.h>  // AVR 레지스터 및 함수 사용을 위한 헤더 파일
#include <util/delay.h>  // 딜레이 함수 사용을 위한 헤더 파일
#include "UART0.h"  // 사용자 정의 UART0 라이브러리 헤더 파일
#include <string.h>  // 문자열 관련 함수 사용을 위한 헤더 파일
#include <stdio.h>  // 입출력 함수 사용을 위한 헤더 파일
#include <avr/interrupt.h>  // 인터럽트 관련 함수 사용을 위한 헤더 파일

FILE OUTPUT = FDEV_SETUP_STREAM(UART0_transmit, NULL, _FDEV_SETUP_WRITE);  // 시리얼 출력 스트림 설정
FILE INPUT = FDEV_SETUP_STREAM(NULL, UART0_receive, _FDEV_SETUP_READ);  // 시리얼 입력 스트림 설정

volatile unsigned int adc_value;  // ADC 변환 결과를 저장할 변수

void ADC_init(unsigned char channel)
{
    ADMUX |= (1<<REFS0);  // ADC의 참조전압을 AVCC로 설정
    ADCSRA |= 0x07;  // ADC 프리스케일러 값을 128로 설정
    ADCSRA |= (1<<ADEN);  // ADC 활성화
    ADCSRA |= (1<<ADFR);  // 자동 트리거 모드 설정
    ADCSRA |= (1<<ADIE);  // ADC 변환 완료 인터럽트 활성화
    ADMUX = ((ADMUX & 0xE0) | channel);  // 입력 채널 설정
    ADCSRA |= (1<<ADSC);  // ADC 변환 시작
    sei();  // 전역 인터럽트 활성화
}

ISR(ADC_vect)
{
    adc_value = ADC;  // ADC 변환 결과를 저장
}

int main(void)
{
    stdout = &OUTPUT;  // 표준 출력 스트림을 시리얼 출력으로 설정
    stdin = &INPUT;  // 표준 입력 스트림을 시리얼 입력으로 설정
    UART0_init();  // UART0 초기화 함수 호출
    ADC_init(1);  // ADC 초기화 함수 호출 (입력 채널 1 설정)
    DDRA = 0xff;  // 포트A를 출력으로 설정
    PORTA = 0x00;  // 포트A의 출력 초기화

    while (1)
    {
        float input_volt = (adc_value * 5.0) / 1024.0;  // ADC 값을 전압으로 변환
        printf("%f\r\n", input_volt);  // 시리얼 통신을 통해 전압 값 출력

        if (input_volt >= 3)
            PORTA = 0xFF;  // 전압이 3V 이상일 경우 포트A의 출력을 1로 설정
        else
            PORTA = 0x00;  // 전압이 3V 미만일 경우 포트A의 출력을 0으로 설정

        _delay_ms(500);  // 0.5초 딜레이
    }
}
~~~

## 빛감지 둘다


~~~

#define F_CPU 16000000UL
#include <avr/io.h>
#include <util/delay.h>
#include "UART0.h"
#include <string.h>
#include <stdio.h>

// 표준 입출력 스트림 설정
FILE OUTPUT = FDEV_SETUP_STREAM(UART0_transmit, NULL, _FDEV_SETUP_WRITE);
FILE INPUT = FDEV_SETUP_STREAM(NULL, UART0_receive, _FDEV_SETUP_READ);

// ADC 초기화 함수 정의
void ADC_init()
{
    // ADC 참조 전압 설정 (AVCC 사용)
    ADMUX |= (1 << REFS0);

    // ADC 프리스케일러 분주비 설정 (Prescaler = 128)
    ADCSRA |= 0x07;

    // ADC 활성화
    ADCSRA |= (1 << ADEN);
}

// ADC로부터 값 읽는 함수 정의
int read_ADC(unsigned char channel)
{
    // 채널 설정
    ADMUX = ((ADMUX & 0xE0) | channel);

    // ADC 변환 시작
    ADCSRA |= (1 << ADSC);

    // 변환 완료 대기
    while (!(ADCSRA & (1 << ADIF)));

    // ADC 값 반환
    return ADC;
}

int main(void)
{
    // 표준 입출력 스트림 설정
    stdout = &OUTPUT;
    stdin = &INPUT;

    int read;
    UART0_init();
    ADC_init();

    while (1)
    {
        // ADC0 값 읽고 출력
        read = read_ADC(0);
        printf("ADC0: %d\r\n", read);

        // 1초 대기
        _delay_ms(1000);

        // ADC1 값 읽고 출력
        read = read_ADC(1);
        printf("ADC1: %d\r\n", read);

        // 1초 대기
        _delay_ms(1000);
    }
}


# 숫자 표시

## 오버플로우
~~~
#define F_CPU 16000000UL
#include <avr/io.h>
#include <avr/interrupt.h>

// 인터럽트 발생 횟수를 저장할 변수
volatile int interrupt_count = 0;

// 타이머 0 오버플로우 인터럽트 핸들러
ISR(TIMER0_OVF_vect)
{
    interrupt_count++;
}

int main(void)
{
    // 7세그먼트 디스플레이에 숫자를 나타내기 위한 패턴 배열
    uint8_t numbers[] = {0x3F, 0x06, 0x5B, 0x4F, 0x66, 0x6D, 0x7D, 0x27, 0x7F, 0x67};
    
    int count = 0;
    
    // 포트 설정
    DDRC = 0xFF;  // 7세그먼트 LED 포트 (출력)
    DDRG = 0x0F;  // 7세그먼트 선택 포트 (출력)
    PORTG = 0x01; // 초기에 FND0 선택
    
    // 타이머 0 설정
    TCCR0 |= (1 << CS02) | (1 << CS01) | (1 << CS00); // 1024분주
    TIMSK |= (1 << TOIE0); // 타이머 0 오버플로우 인터럽트 활성화

    // 무한 루프
    while (1)
    {
        if (interrupt_count > 64)
        {
            interrupt_count = 0;
            count = (count + 1) % 10;
            PORTC = numbers[count]; // 현재 숫자에 해당하는 패턴 출력
        }
    }
}
~~~